# Первое решение, Титаник VS Айсберг

В первом решении мы предлагаем использовать структуру, называемую friendly-триплетом, которая состоит из эго-графа, вершины-1 и вершины-2. Для формирования обучающего и валидационного датасетов мы случайным образом удаляем некоторые ребра из эго-графов и формируем триплеты. Затем мы применяем модифицированную графовую нейронную сеть, основанную на библиотеках DGL и PyTorch, для обучения и предсказания интенсивности взаимодействия между друзьями по friendly-триплету.

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
import dgl
import torch 

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Считывание данных

In [2]:
# считаем train
train_ds = pd.read_csv('train.csv')

In [3]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122280372 entries, 0 to 122280371
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ego_id  int64  
 1   u       int64  
 2   v       int64  
 3   t       float64
 4   x1      float64
 5   x2      float64
 6   x3      float64
dtypes: float64(4), int64(3)
memory usage: 6.4 GB


In [4]:
train_ds.describe()

,ego_id,u,v,t,x1,x2,x3
count,1.222804e+08,1.222804e+08,1.222804e+08,1.035632e+08,1.222804e+08,1.222804e+08,1.222804e+08
mean,8.508872e+11,8.766378e+01,9.480200e+01,1.140129e+02,6.909837e-01,1.986819e-01,1.526350e-01
std,4.958312e+11,6.990831e+01,7.397358e+01,1.125724e+02,1.354156e+00,7.843783e-01,3.596353e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.209068e+11,3.000000e+01,3.400000e+01,2.760000e+01,1.057975e-05,0.000000e+00,0.000000e+00
50%,8.504035e+11,7.300000e+01,7.900000e+01,8.110000e+01,7.433163e-02,0.000000e+00,0.000000e+00
75%,1.279900e+12,1.330000e+02,1.430000e+02,1.611000e+02,8.960580e-01,0.000000e+00,0.000000e+00
max,1.709397e+12,2.990000e+02,2.990000e+02,5.945000e+02,4.728715e+01,1.074882e+01,1.000000e+00


In [5]:
# считаем test
test_ds = pd.read_csv('test.csv')

In [6]:
# считаем submission
subm_ds = pd.read_csv('submission.csv')

In [7]:
subm_ds

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857
...,...,...,...,...
810971,1709396984676,73,23,1.562170
810972,1709396984676,74,68,0.454210
810973,1709396984676,77,28,0.078504
810974,1709396984676,79,38,0.981812


In [8]:
# считаем attr
attr_ds = pd.read_csv('attr.csv')

In [9]:
attr_ds 

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1
...,...,...,...,...,...,...,...
14930743,1709396984692,2,16,492149712,2,769209871,-1
14930744,1709396984692,12,15,-1,1,-1,-1
14930745,1709396984692,18,23,-1,1,-1,-1
14930746,1709396984692,4,16,650683235,1,-1,-1


In [10]:
# заметим, что в test есть ребра из submission. Это плохо, запомним, какие именно это ребра
nan_edges = np.asarray(subm_ds[["ego_id", "u", "v"]])
nan_edges_dict = {}
for edge_info in nan_edges:
    if edge_info[0] not in nan_edges_dict:
        nan_edges_dict[edge_info[0]] = [[edge_info[1]], [edge_info[2]]]
    else:
        nan_edges_dict[edge_info[0]][0].append(edge_info[1])
        nan_edges_dict[edge_info[0]][1].append(edge_info[2])

## Препроцессинг данных. Представление ego-графа как dgl-графа

In [28]:
train_graph_info = -np.ones( (len(train_ds['ego_id'].unique()), 300, 5) )
ego_ids = train_ds['ego_id'].unique()

In [29]:
# формирование признаков вершин для графов из train

for i in tqdm(range(len(ego_ids))):
    ego_train_ds = train_ds[train_ds['ego_id'] == ego_ids[i]]
    node_set = np.unique(ego_train_ds[['v', 'u']].values)
    data = attr_ds[(attr_ds['ego_id'] == ego_ids[i])]
    for node in node_set:
        node_attr_ds_u = data[(data['u'] == node)]

        if not node_attr_ds_u.empty:
            train_graph_info[i][node] = node_attr_ds_u.values[0,2:]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.47it/s]


In [30]:
test_graph_info = -np.ones( (len(test_ds['ego_id'].unique()), 300, 5) )
ego_ids = test_ds['ego_id'].unique()

In [31]:
# формирование признаков вершин для графов из test

for i in tqdm(range(len(ego_ids))):
    ego_test_ds = test_ds[test_ds['ego_id'] == ego_ids[i]]
    node_set = np.unique(ego_test_ds[['v', 'u']].values)
    data = attr_ds[(attr_ds['ego_id'] == ego_ids[i])]
    for node in node_set:
        node_attr_ds_u = data[(data['u'] == node)]

        if not node_attr_ds_u.empty:
            test_graph_info[i][node] = node_attr_ds_u.values[0,2:]
            
# не стоит верить линейке загрузки 2/2. На самом деле ego_ids больше, просто эта часть кода считалась на другом
# устройстве. На этом устройстве признаки загружались из файлов

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.35it/s]


In [36]:
# np.savetxt("train_np.txt", train_graph_info.reshape((len(train_ds['ego_id'].unique()), 1500)))
# np.savetxt("test_np.txt", test_graph_info.reshape((len(test_ds['ego_id'].unique()), 1500)))
# сохранение признаков для дальнейшего переиспользования на других устройствах

In [41]:
# формирование эго-графов в виде dgl-графов из train
train_graph_dataset = []

ego_ids = train_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = train_ds[train_ds["ego_id"] == ego_ids[i]]
    graph = dgl.graph(data[['u', 'v']].values.tolist(), num_nodes=300)
    x1 = np.array(data[['x1']]).reshape(-1)
    
    graph.ndata["features"] = torch.from_numpy(train_graph_info[i])

    graph.edata["attr_x1"] = torch.from_numpy(x1)

    train_graph_dataset.append(graph)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.03it/s]


In [43]:
# формирование эго-графов в виде dgl-графов из test
test_graph_dataset = []

ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = test_ds[test_ds["ego_id"] == ego_ids[i]]
    graph = dgl.graph(data[['u', 'v']].values.tolist(), num_nodes=300)
    x1 = np.array(data[['x1']]).reshape(-1)
    
    graph.ndata["features"] = torch.from_numpy(test_graph_info[i])

    graph.edata["attr_x1"] = torch.from_numpy(x1)

    test_graph_dataset.append(graph)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 33.71it/s]


In [44]:
# dgl.data.utils.save_graphs("./train_data.bin", train_graph_dataset)
# dgl.data.utils.save_graphs("./test_data.bin", test_graph_dataset)
# сохранение графов для дальнейшего переиспользования на других устройствах

### Обработка nan в признаках ребер графа

In [75]:
# обработка nan-значений в признаках и добавление признаков ребер в train
ego_ids = train_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = train_ds[train_ds["ego_id"] == ego_ids[i]]
    edge_feat = np.array(data[['x2', 'x3', 't']]).reshape(len(data), 3)
    train_graph_dataset[i].edata["edge_feat"] = torch.nan_to_num(torch.from_numpy(edge_feat))
    train_graph_dataset[i].edata["attr_x1"] = torch.nan_to_num(train_graph_dataset[i].edata["attr_x1"])
    

100%|██████████████████████████████████████████████████████████████████████████| 61786/61786 [1:23:42<00:00, 12.30it/s]


In [77]:
# обработка nan-значений в признаках и добавление признаков ребер в test
ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = test_ds[test_ds["ego_id"] == ego_ids[i]]
    edge_feat = np.array(data[['x2', 'x3', 't']]).reshape(len(data), 3)
    test_graph_dataset[i].edata["edge_feat"] = torch.nan_to_num(torch.from_numpy(edge_feat))
    test_graph_dataset[i].edata["attr_x1"] = torch.nan_to_num(test_graph_dataset[i].edata["attr_x1"])

100%|████████████████████████████████████████████████████████████████████████████| 20596/20596 [09:48<00:00, 34.98it/s]


In [78]:
# dgl.data.utils.save_graphs("./edge_train_data.bin", train_graph_dataset)
# dgl.data.utils.save_graphs("./edge_test_data.bin", test_graph_dataset)
# сохранение графов для дальнейшего переиспользования на других устройствах

### Обработка признаков вершин графа

In [80]:
import numpy as np
# обработка признаков вершин эго-графов из train: нормализация, пропуски
ego_ids = train_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = train_graph_dataset[i].ndata["features"]
    data[data == -1] = np.nan

    feature1_col = data[:, 0]
    mean_feature1 = np.nanmean(feature1_col)
    feature1_col[np.isnan(feature1_col)] = mean_feature1
    data[:, 0] = feature1_col

    int_columns = [1, 3, 4]
    for col in int_columns:
        feature_col = data[:, col]
        feature_col[np.isnan(feature_col)] = 0
        
        mean = np.nanmean(feature_col)
        std = np.nanstd(feature_col)
        feature_col = (feature_col - mean) / std
        data[:, col] = feature_col

    feature3_col = data[:, 2]
    mean_feature3 = np.nanmean(feature3_col)
    feature3_col[np.isnan(feature3_col)] = mean_feature3
    data[:, 2] = feature3_col
    
    train_graph_dataset[i].ndata["features"] = data

  0%|                                                                                        | 0/61786 [00:00<?, ?it/s]C:\Users\daled\AppData\Local\Temp\ipykernel_24856\855138049.py:12: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)
  feature1_col[np.isnan(feature1_col)] = mean_feature1
C:\Users\daled\AppData\Local\Temp\ipykernel_24856\855138049.py:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)
  feature_col[np.isnan(feature_col)] = 0
C:\Users\daled\AppData\Local\Temp\ipykernel_24856\855138049.py:29: Us

In [82]:
train_graph_dataset[0].ndata["features"]

tensor([[43.0000,  0.0998,  1.0000, -0.8492, -0.7828],
        [39.8551,  0.0998,  2.0000,  1.4857,  1.8303],
        [39.0000,  0.0998,  1.0000,  1.5229,  0.5114],
        ...,
        [29.0000, -1.2736,  2.0000, -0.8492, -0.7828],
        [48.0000,  1.7937,  2.0000, -0.8492, -0.7828],
        [48.0000,  1.7146,  2.0000, -0.8492, -0.7828]], dtype=torch.float64)

In [84]:
import numpy as np
# обработка признаков вершин эго-графов из test: нормализация, пропуски

ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    data = test_graph_dataset[i].ndata["features"]
    data[data == -1] = np.nan

    feature1_col = data[:, 0]
    mean_feature1 = np.nanmean(feature1_col)
    feature1_col[np.isnan(feature1_col)] = mean_feature1

    int_columns = [1, 3, 4]
    for col in int_columns:
        feature_col = data[:, col]
        feature_col[np.isnan(feature_col)] = 0
                
        mean = np.nanmean(feature_col)
        std = np.nanstd(feature_col)
        feature_col = (feature_col - mean) / std
        data[:, col] = feature_col

    feature3_col = data[:, 2]
    mean_feature3 = np.nanmean(feature3_col)
    feature3_col[np.isnan(feature3_col)] = mean_feature3
    
    test_graph_dataset[i].ndata["features"] = data

  0%|                                                                                        | 0/20596 [00:00<?, ?it/s]C:\Users\daled\AppData\Local\Temp\ipykernel_24856\2912941494.py:12: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)
  feature1_col[np.isnan(feature1_col)] = mean_feature1
C:\Users\daled\AppData\Local\Temp\ipykernel_24856\2912941494.py:18: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)
  feature_col[np.isnan(feature_col)] = 0
C:\Users\daled\AppData\Local\Temp\ipykernel_24856\2912941494.py:28:

In [85]:
# dgl.data.utils.save_graphs("./new_train_data.bin", train_graph_dataset)
# dgl.data.utils.save_graphs("./new_test_data.bin", test_graph_dataset)
# сохранение графов для дальнейшего переиспользования на других устройствах

In [11]:
import dgl
test_graph_dataset = dgl.data.utils.load_graphs("./new_test_data.bin")
test_graph_dataset = test_graph_dataset[0]

train_graph_dataset = dgl.data.utils.load_graphs("./new_train_data.bin")
train_graph_dataset = train_graph_dataset[0]
# загрузка графов из ранее сохраненных графов в файл

In [12]:
# удаление из test ребер из submission, о которых упоминалось ранее
ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    if ego_ids[i] in nan_edges_dict:
        test_graph_dataset[i] = dgl.remove_edges(test_graph_dataset[i], test_graph_dataset[i].edge_ids(*nan_edges_dict[ego_ids[i]]))

100%|███████████████████████████████████████████████████████████████████████████| 20596/20596 [00:48<00:00, 421.57it/s]


### Формирование датасетов train/validation, состоящих из friendly-триплетов

In [13]:
print(len(ego_ids))

20596


In [20]:
train_graphs = []
train_vertex_a = []
train_vertex_b = []
train_targets = []


# план удалить из графа рандомные ребра
# добавить их в датасет как friendly-триплет в виде граф-вершина_src-вершина_dst
# важно: мы формируем сэмплы и из test, и из train, так как хотим выучить и графы, 
# на которых будет формироваться предсказание
ego_ids = train_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    g = train_graph_dataset[i]

    num_edges = g.number_of_edges()
    # удаляем ребер

    removed_edge_indices_max = np.argpartition(g.edata["attr_x1"], -3)[-3:]
    removed_edge_indices_min = np.argpartition(-g.edata["attr_x1"], -3)[-3:]
    removed_edge_indices = np.concatenate([removed_edge_indices_min, removed_edge_indices_max])
    
    # удаляем выбранные ребра из графа
    graph_removed_edges = dgl.remove_edges(g, removed_edge_indices)

    src_edges = np.take(g.edges()[0].numpy(), removed_edge_indices)
    dst_edges = np.take(g.edges()[1].numpy(), removed_edge_indices)
    
    for i in range(len(removed_edge_indices)):
        src, dst = src_edges[i], dst_edges[i]
        train_graphs.append(graph_removed_edges)
        train_vertex_a.append(src.item())
        train_vertex_b.append(dst.item())

    train_targets += np.take(g.edata["attr_x1"], removed_edge_indices).tolist()


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 40.61it/s]


In [22]:
ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    g = test_graph_dataset[i]

    num_edges = g.number_of_edges()
    # удаляем ребер

    removed_edge_indices_max = np.argpartition(g.edata["attr_x1"], -10)[-10:]
    removed_edge_indices_min = np.argpartition(-g.edata["attr_x1"], -10)[-10:]
    removed_edge_indices = np.concatenate([removed_edge_indices_min, removed_edge_indices_max])

    src_edges = np.take(g.edges()[0].numpy(), removed_edge_indices)
    dst_edges = np.take(g.edges()[1].numpy(), removed_edge_indices)
    graph_removed_edges = dgl.remove_edges(g, removed_edge_indices)

    
    for i in range(len(removed_edge_indices)):
        src, dst = src_edges[i], dst_edges[i]
        train_graphs.append(graph_removed_edges)
        train_vertex_a.append(src.item())
        train_vertex_b.append(dst.item())

    train_targets += np.take(g.edata["attr_x1"], removed_edge_indices).tolist()
    test_graph_dataset[i] = graph_removed_edges

100%|███████████████████████████████████████████████████████████████████████████| 20596/20596 [00:25<00:00, 796.28it/s]


In [23]:
from sklearn.model_selection import train_test_split
# делим на train и validation
graphs_train, graphs_valid, vertex_a_train, vertex_a_valid, vertex_b_train, vertex_b_valid, targets_train, targets_valid = train_test_split(train_graphs, 
                                                                                                                                            train_vertex_a,
                                                                                                                                            train_vertex_b, 
                                                                                                                                            train_targets, 
                                                                                                                                            test_size=0.1,
                                                                                                                                            random_state=42)

In [24]:
from torch.utils.data import Dataset
import dgl

class GraphInteractionDataset(Dataset):
    def __init__(self, graphs, vertex_a, vertex_b, targets):
        self.graphs = graphs
        self.vertex_a = vertex_a
        self.vertex_b = vertex_b
        self.targets = targets

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, index):
        graph = self.graphs[index]
        a = self.vertex_a[index]
        b = self.vertex_b[index]
        target = self.targets[index]
        return graph, a, b, target

def collate_fn(batch):
    graphs, vertex_a, vertex_b, targets = zip(*batch)
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(vertex_a), torch.tensor(vertex_b), torch.tensor(targets)

# класс для формирования DataLoader

train_dataset = GraphInteractionDataset(graphs_train, vertex_a_train, vertex_b_train, targets_train)
valid_dataset = GraphInteractionDataset(graphs_valid, vertex_a_valid, vertex_b_valid, targets_valid)

### Написать модельку и обучить

In [25]:
import torch

In [26]:
from torch.utils.data import DataLoader
# создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [27]:
len(train_loader)

204538

In [28]:
len(valid_loader)

22727

In [29]:
import torch
import dgl.nn as dglnn
import torch.nn as nn
import dgl.function as fn
import torch.optim as optim
from dgl.nn import GraphConv

class GraphPredictor(torch.nn.Module):
    def __init__(self, node_input_dim, edge_input_dim, node_hidden_dim, edge_hidden_dim):
        super(GraphPredictor, self).__init__()
        self.node_encoder = nn.Sequential(
            torch.nn.Linear(node_input_dim, node_hidden_dim),
            torch.nn.ReLU()
        )
        self.edge_encoder = nn.Sequential(
            torch.nn.Linear(edge_input_dim, edge_hidden_dim),
            torch.nn.ReLU()
        )
        self.graph_encoder = dglnn.GraphConv(node_hidden_dim, edge_hidden_dim)

        self.linear = nn.Linear(3 * node_hidden_dim, 1)
        
    def forward(self, graph, src, dst):
        graph = dgl.add_self_loop(graph)
        node_feat = torch.nan_to_num(graph.ndata['features'].float())
        node_feat = self.node_encoder(node_feat)
        edge_feat = self.edge_encoder(torch.nan_to_num(graph.edata['edge_feat'].float()))
        
        graph.ndata['features'] = node_feat
        graph.edata['edge_feat'] = edge_feat

        graph.update_all(message_func=fn.copy_e('edge_feat', 'm'), reduce_func=fn.sum('m', 'neigh'))
        graph.ndata['hidden'] = graph.ndata['features'] + graph.ndata['neigh']
        h = graph.ndata['hidden']
        graph.ndata['features'] = h
        # хотим получать эмбеддинги с нужными индексами, причем каждый номер это номер вершины + номер графа * 300
#         print(graph_emb)
        src_emb = h[src]
        dst_emb = h[dst]
        graph_emb = torch.tile(torch.mean(h, dim = 0), (src_emb.shape[0], 1))

        
        emb_concat = torch.cat([graph_emb, src_emb, dst_emb], dim=1)
        
        output = self.linear(emb_concat)
        return output.reshape(-1)
    
device = torch.device('cuda')
model = GraphPredictor(5, 3, 8, 8).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
num_epochs = 3
best_loss = float('inf')
best_model_state_dict = None

model.train()
for epoch in tqdm(range(num_epochs)):
    running_loss = 0
    running_val_loss = 0
    num = 0
    
    for graph, a, b, target in train_loader:
        if num % 100 == 0:
            print(num, "/" , len(train_loader))

        num += 1
        graph.ndata['features']  = graph.ndata['features'] / np.linalg.norm(graph.ndata['features'])
        a = torch.tensor([a[i] + 300 * i for i in range(len(a))])
        b = torch.tensor([b[i] + 300 * i for i in range(len(b))])
        graph = graph.to(device)
        a = a.to(device)
        b = b.to(device)
        target = target.to(device)
                
        optimizer.zero_grad()
        output = model(graph, a, b)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1} / {num_epochs}, Loss: {average_loss:.4f}")
    num = 0

    model.eval()
    with torch.no_grad():
        for graph, a, b, target in valid_loader:
            if num % 100 == 0:
                print(num, "/" , len(valid_loader))

            num += 1
            graph.ndata['features'] = graph.ndata['features'] / np.linalg.norm(graph.ndata['features'])

            a = torch.tensor([a[i] + 300 * i for i in range(len(a))])
            b = torch.tensor([b[i] + 300 * i for i in range(len(b))])
            graph = graph.to(device)
            a = a.to(device)
            b = b.to(device)
            target = target.to(device)

            output = model(graph, a, b)
            val_loss = criterion(output, target)
            running_val_loss += val_loss.item()
        average_val_loss = running_val_loss / len(valid_loader)
        print(f"Epoch {epoch+1} / {num_epochs}, Validation Loss: {average_val_loss:.4f}")
        
        if average_val_loss < best_loss:
            best_loss = average_val_loss
            best_model_state_dict = model.state_dict()


# план - модель на вход принимает граф + пару вершин, и гоняет через графовую свертку весь граф
# потом из пары фичей вершин нужных предикт
# тут датасет имеет вид friendly-триплетов, предскажем коэффициент


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

0 / 204538


C:\Users\daled\anaconda3\lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


100 / 204538
200 / 204538
300 / 204538
400 / 204538
500 / 204538
600 / 204538
700 / 204538
800 / 204538
900 / 204538
1000 / 204538
1100 / 204538
1200 / 204538
1300 / 204538
1400 / 204538
1500 / 204538
1600 / 204538
1700 / 204538
1800 / 204538
1900 / 204538
2000 / 204538
2100 / 204538
2200 / 204538
2300 / 204538
2400 / 204538
2500 / 204538
2600 / 204538
2700 / 204538
2800 / 204538
2900 / 204538
3000 / 204538
3100 / 204538
3200 / 204538
3300 / 204538
3400 / 204538
3500 / 204538
3600 / 204538
3700 / 204538
3800 / 204538
3900 / 204538
4000 / 204538
4100 / 204538
4200 / 204538
4300 / 204538
4400 / 204538
4500 / 204538
4600 / 204538
4700 / 204538
4800 / 204538
4900 / 204538
5000 / 204538
5100 / 204538
5200 / 204538
5300 / 204538
5400 / 204538
5500 / 204538
5600 / 204538
5700 / 204538
5800 / 204538
5900 / 204538
6000 / 204538
6100 / 204538
6200 / 204538
6300 / 204538
6400 / 204538
6500 / 204538
6600 / 204538
6700 / 204538
6800 / 204538
6900 / 204538
7000 / 204538
7100 / 204538
7200 / 204538
7

55500 / 204538
55600 / 204538
55700 / 204538
55800 / 204538
55900 / 204538
56000 / 204538
56100 / 204538
56200 / 204538
56300 / 204538
56400 / 204538
56500 / 204538
56600 / 204538
56700 / 204538
56800 / 204538
56900 / 204538
57000 / 204538
57100 / 204538
57200 / 204538
57300 / 204538
57400 / 204538
57500 / 204538
57600 / 204538
57700 / 204538
57800 / 204538
57900 / 204538
58000 / 204538
58100 / 204538
58200 / 204538
58300 / 204538
58400 / 204538
58500 / 204538
58600 / 204538
58700 / 204538
58800 / 204538
58900 / 204538
59000 / 204538
59100 / 204538
59200 / 204538
59300 / 204538
59400 / 204538
59500 / 204538
59600 / 204538
59700 / 204538
59800 / 204538
59900 / 204538
60000 / 204538
60100 / 204538
60200 / 204538
60300 / 204538
60400 / 204538
60500 / 204538
60600 / 204538
60700 / 204538
60800 / 204538
60900 / 204538
61000 / 204538
61100 / 204538
61200 / 204538
61300 / 204538
61400 / 204538
61500 / 204538
61600 / 204538
61700 / 204538
61800 / 204538
61900 / 204538
62000 / 204538
62100 / 20

...

 67%|█████████████████████████████████████████████████████▎                          | 2/3 [1:07:14<33:33, 2013.18s/it]

22700 / 22727
Epoch 2 / 3, Validation Loss: 12.5319
0 / 204538
100 / 204538
200 / 204538


 67%|█████████████████████████████████████████████████████▎                          | 2/3 [1:07:17<33:38, 2018.62s/it]

300 / 204538


KeyboardInterrupt: 

In [30]:
model.load_state_dict(best_model_state_dict)

<All keys matched successfully>

In [ ]:
# посчитать с фичами ребер

In [31]:
test_graphs = []
test_vertex_a = []
test_vertex_b = []
test_targets = []

ego_ids = test_ds['ego_id'].unique()

for i in tqdm(range(len(ego_ids))):
    if ego_ids[i] in nan_edges_dict:
        vertex = nan_edges_dict[ego_ids[i]]
        for j in range(len(vertex[0])):
            test_graphs.append(test_graph_dataset[i])
            test_vertex_a.append(vertex[0][j])
            test_vertex_b.append(vertex[1][j])
            test_targets.append(0)

100%|█████████████████████████████████████████████████████████████████████████| 20596/20596 [00:00<00:00, 64237.45it/s]


In [32]:
test_dataset = GraphInteractionDataset(test_graphs, test_vertex_a, test_vertex_b, test_targets)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
result = []
num = 0

with torch.no_grad():
    for graph, a, b, target in tqdm(test_loader):
#         if num % 200 == 0:
#             print(num, "/" , len(test_loader))
        num += 1
        graph.ndata['features'] = graph.ndata['features'] / np.linalg.norm(graph.ndata['features'])
        
        a = torch.tensor([a[i] + 300 * i for i in range(len(a))])
        b = torch.tensor([b[i] + 300 * i for i in range(len(b))])
        graph = graph.to(device)
        a = a.to(device)
        b = b.to(device)

        output = model(graph, a, b)
        result += output.tolist()

print(result)

100%|█████████████████████████████████████████████████████████████████████████| 101372/101372 [14:32<00:00, 116.17it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
subm_ds["x1"] = result
subm_ds.to_csv("subm.csv", index=False)

In [34]:
subm_ds.tail(20)

,ego_id,u,v,x1
810956,1709396984676,3,0,2.981724
810957,1709396984676,6,38,6.259113
810958,1709396984676,15,14,2.746050
810959,1709396984676,18,11,5.763816
810960,1709396984676,25,32,8.662182
810961,1709396984676,29,20,6.084879
810962,1709396984676,29,49,6.120730
810963,1709396984676,30,8,8.145971
810964,1709396984676,32,8,10.917710
810965,1709396984676,33,91,3.710418


In [35]:
subm_ds = pd.read_csv('subm.csv')

In [36]:
subm_ds

,ego_id,u,v,x1
0,8,0,93,2.283422
1,8,0,143,2.283422
2,8,0,151,2.487226
3,8,1,24,2.277049
4,8,5,4,2.277049
...,...,...,...,...
810971,1709396984676,73,23,2.842059
810972,1709396984676,74,68,3.000031
810973,1709396984676,77,28,2.842059
810974,1709396984676,79,38,6.994520
